<a href="https://colab.research.google.com/github/mohammadzavvari/LIDC-IDRI-DICOM-/blob/main/LIDC_IDRI_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pylidc

In [ ]:
import numpy as np
import pandas as pd

import torch.nn as nn
import torch


In [ ]:
import pydicom as dicom
import pylidc as pl

In [ ]:
!unzip -f ./LIDC-XML-only.zip

Archive:  ./LIDC-XML-only.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ./LIDC-XML-only.zip or
        ./LIDC-XML-only.zip.zip, and cannot find ./LIDC-XML-only.zip.ZIP, period.


In [ ]:
?pydicom

In [ ]:
?pylidc

In [ ]:
scans = pl.query(pl.Scan).filter(pl.Scan.slice_thickness <= 1,
                                 pl.Scan.pixel_spacing <= 0.6)
print(scans.count())
# => 31

31


In [ ]:
pid = 'LIDC-IDRI-0078'
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()

In [ ]:
type(scan)

pylidc.Scan.Scan

In [ ]:
print(scan)

Scan(id=1,patient_id=LIDC-IDRI-0078)


In [ ]:
?pylidc.Scan.Scan()

Object `pylidc.Scan.Scan()` not found.


In [ ]:
len(scan.annotations)

13

In [ ]:
scan.annotations

[Annotation(id=1,scan_id=1),
 Annotation(id=2,scan_id=1),
 Annotation(id=3,scan_id=1),
 Annotation(id=4,scan_id=1),
 Annotation(id=5,scan_id=1),
 Annotation(id=6,scan_id=1),
 Annotation(id=7,scan_id=1),
 Annotation(id=8,scan_id=1),
 Annotation(id=9,scan_id=1),
 Annotation(id=10,scan_id=1),
 Annotation(id=11,scan_id=1),
 Annotation(id=12,scan_id=1),
 Annotation(id=13,scan_id=1)]

In [ ]:
type(scan.annotations[0])

pylidc.Annotation.Annotation

In [ ]:
nods = scan.cluster_annotations()

print("%s has %d nodules." % (scan, len(nods)))
# => Scan(id=1,patient_id=LIDC-IDRI-0078) has 4 nodules.

for i,nod in enumerate(nods):
    print("Nodule %d has %d annotations." % (i+1, len(nods[i])))
# => Nodule 1 has 4 annotations.
# => Nodule 2 has 4 annotations.
# => Nodule 3 has 1 annotations.
# => Nodule 4 has 4 annotations.

Scan(id=1,patient_id=LIDC-IDRI-0078) has 4 nodules.
Nodule 1 has 4 annotations.
Nodule 2 has 4 annotations.
Nodule 3 has 1 annotations.
Nodule 4 has 4 annotations.


In [ ]:
nods

[[Annotation(id=2,scan_id=1),
  Annotation(id=6,scan_id=1),
  Annotation(id=10,scan_id=1),
  Annotation(id=13,scan_id=1)],
 [Annotation(id=1,scan_id=1),
  Annotation(id=5,scan_id=1),
  Annotation(id=9,scan_id=1),
  Annotation(id=12,scan_id=1)],
 [Annotation(id=8,scan_id=1)],
 [Annotation(id=3,scan_id=1),
  Annotation(id=4,scan_id=1),
  Annotation(id=7,scan_id=1),
  Annotation(id=11,scan_id=1)]]

In [ ]:
vol = scan.to_volume()
print(vol.shape)

Loading dicom files ... This may take a moment.


/usr/local/lib/python3.7/dist-packages/pylidc/Scan.py:67: UserWarning: Could not find `dicom` configuration section or  `path` configuration option under that section.A template config file will be written to /root/.pylidcrc.
  warnings.warn(msg.format(conf_file))


RuntimeError: ignored

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as manim
from skimage.measure import find_contours

import pylidc as pl
from pylidc.utils import consensus


# Query for a scan, and convert it to an array volume.
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == 'LIDC-IDRI-0078').first()
vol = scan.to_volume()

# Cluster the annotations for the scan, and grab one.
nods = scan.cluster_annotations()
anns = nods[0]

# Perform a consensus consolidation and 50% agreement level.
# We pad the slices to add context for viewing.
cmask,cbbox,masks = consensus(anns, clevel=0.5,
                              pad=[(20,20), (20,20), (0,0)])

# Get the central slice of the computed bounding box.
k = int(0.5*(cbbox[2].stop - cbbox[2].start))

# Set up the plot.
fig,ax = plt.subplots(1,1,figsize=(5,5))
ax.imshow(vol[cbbox][:,:,k], cmap=plt.cm.gray, alpha=0.5)

# Plot the annotation contours for the kth slice.
colors = ['r', 'g', 'b', 'y']
for j in range(len(masks)):
    for c in find_contours(masks[j][:,:,k].astype(float), 0.5):
        label = "Annotation %d" % (j+1)
        plt.plot(c[:,1], c[:,0], colors[j], label=label)

# Plot the 50% consensus contour for the kth slice.
for c in find_contours(cmask[:,:,k].astype(float), 0.5):
    plt.plot(c[:,1], c[:,0], '--k', label='50% Consensus')

ax.axis('off')
ax.legend()
plt.tight_layout()
#plt.savefig("../images/consensus.png", bbox_inches="tight")
plt.show()

Loading dicom files ... This may take a moment.


RuntimeError: ignored

In [ ]:
!mkdir /usr/share/desktop-directories/
!wget -P ~/NBIA-Data-Retriever https://cbiit-download.nci.nih.gov/nbia/releases/ForTCIA/NBIADataRetriever_4.2/nbia-data-retriever-4.2.deb
!dpkg -i ~/NBIA-Data-Retriever/nbia-data-retriever-4.2.deb

In [ ]:
!wget https://wiki.cancerimagingarchive.net/download/attachments/44499647/LIDC-IDRI-StandardizedRepresentation-March2020-manifest.tcia?version=1&modificationDate=1585445399387&api=v2

In [ ]:
!/opt/nbia-data-retriever/nbia-data-retriever --cli /content/LIDC-IDRI-StandardizedRepresentation-March2020-manifest.tcia -d /content/ -v

In [ ]:
!zip -r lidc.zip /content/LIDC-IDRI-StandardizedRepresentation-March2020-manifest


In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

!cp lidc.zip /content/drive/MyDrive/lidc-backup.zip

In [25]:
!touch /home/mohammad/pylidcrc

In [26]:
!cp /home/mohammad/pylidcrc /home/mohammad/.pylidcrc
!cp /home/mohammad/.pylidcrc /root/.pylidcrc

In [16]:
!ls -a /home/mohammad/

.  ..  pylidcrc  .pylidcrc  .pylidcrc.swp


In [27]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as manim
from skimage.measure import find_contours

import pylidc as pl
from pylidc.utils import consensus


# Query for a scan, and convert it to an array volume.
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == 'LIDC-IDRI-0078').first()
vol = scan.to_volume()

# Cluster the annotations for the scan, and grab one.
nods = scan.cluster_annotations()
anns = nods[0]

# Perform a consensus consolidation and 50% agreement level.
# We pad the slices to add context for viewing.
cmask,cbbox,masks = consensus(anns, clevel=0.5,
                              pad=[(20,20), (20,20), (0,0)])

# Get the central slice of the computed bounding box.
k = int(0.5*(cbbox[2].stop - cbbox[2].start))

# Set up the plot.
fig,ax = plt.subplots(1,1,figsize=(5,5))
ax.imshow(vol[cbbox][:,:,k], cmap=plt.cm.gray, alpha=0.5)

# Plot the annotation contours for the kth slice.
colors = ['r', 'g', 'b', 'y']
for j in range(len(masks)):
    for c in find_contours(masks[j][:,:,k].astype(float), 0.5):
        label = "Annotation %d" % (j+1)
        plt.plot(c[:,1], c[:,0], colors[j], label=label)

# Plot the 50% consensus contour for the kth slice.
for c in find_contours(cmask[:,:,k].astype(float), 0.5):
    plt.plot(c[:,1], c[:,0], '--k', label='50% Consensus')

ax.axis('off')
ax.legend()
plt.tight_layout()
#plt.savefig("../images/consensus.png", bbox_inches="tight")
plt.show()

Loading dicom files ... This may take a moment.


OSError: ignored

In [28]:
!cp /content/NBIADataRetrieverCLI-20220726080705.log /content/drive/MyDrive/NBIADataRetrieverCLI-20220726080705.log

In [29]:

# Query for a scan, and convert it to an array volume.
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == 'LIDC-IDRI-0078').first()
vol = scan.to_volume()

Loading dicom files ... This may take a moment.


OSError: ignored